## Importing libraries

In [11]:
import pandas as pd
import random
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier

## Data Collection

In [12]:
df=pd.read_csv('WA_Fn-UseC_-HR-Employee-Attrition.csv')
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


## Feature Engineering & NLP Sentiment Analysis

In [13]:
# A list of mock negative/neutral comments
exit_comments = [
    "Manager was unsupportive and micro-managed.",
    "Lack of career growth and clear path forward.",
    "Salary and benefits were not competitive.",
    "Commute was too long and no remote work option.",
    "Company culture felt disconnected from values.",
    "Poor work-life balance."
]

# Create the new columns
df['ExitCommentText'] = None
df['SentimentScore'] = None

for index, row in df.iterrows():
    if row['Attrition'] == 'Yes':
        comment = random.choice(exit_comments)
        df.at[index, 'ExitCommentText'] = comment
        sentiment = TextBlob(comment).sentiment.polarity
        df.at[index, 'SentimentScore'] = sentiment

# Fill non-attrition employees with a neutral 0
df['SentimentScore'] = df['SentimentScore'].fillna(0)

C:\Users\amitm\AppData\Local\Temp\ipykernel_5632\1319504150.py:23: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['SentimentScore'] = df['SentimentScore'].fillna(0)


## Data Preprocessing & Feature Encoding

In [14]:
from sklearn.preprocessing import LabelEncoder

# 1. Create the numeric target variable (y)
df['Attrition_numeric'] = df['Attrition'].apply(lambda x: 1 if x == 'Yes' else 0)
y = df['Attrition_numeric']

# 2. Define the feature DataFrame (X) by dropping all non-feature columns
# These are: identifiers, the target (and its original), 
# constant columns, and the raw text column.
X = df.drop(
    ['Attrition_numeric', 'EmployeeNumber', 'EmployeeCount', 'StandardHours', 'Over18', 'ExitCommentText'], 
    axis=1, 
    errors='ignore'
)

# 3. Now, get dummies ONLY on the clean feature DataFrame 'X'
X_processed = pd.get_dummies(X, drop_first=True)

# 4. Handle any remaining missing values (if any)
X_processed = X_processed.fillna(0)

# 5. Rename for consistency with the next step
X = X_processed

print("Step 4 preprocessing complete. X features are ready.")
print(X.head())

Step 4 preprocessing complete. X features are ready.
   Age  DailyRate  DistanceFromHome  Education  EnvironmentSatisfaction  \
0   41       1102                 1          2                        2   
1   49        279                 8          1                        3   
2   37       1373                 2          2                        4   
3   33       1392                 3          4                        4   
4   27        591                 2          1                        1   

   HourlyRate  JobInvolvement  JobLevel  JobSatisfaction  MonthlyIncome  ...  \
0          94               3         2                4           5993  ...   
1          61               2         2                2           5130  ...   
2          92               2         1                3           2090  ...   
3          56               3         1                3           2909  ...   
4          40               3         1                2           3468  ...   

   JobRole_Labo

## Model Training & Generating Flight Risk Scores

In [15]:
# Initialize and train the model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X, y)

# Predict the probability of attrition
probabilities = rf_model.predict_proba(X)

# We only want the probability of '1' (Attrition='Yes')
# This is your "Flight Risk Score"
df['FlightRiskScore'] = probabilities[:, 1]

print(df[['EmployeeNumber', 'Attrition', 'FlightRiskScore', 'SentimentScore']].head())

   EmployeeNumber Attrition  FlightRiskScore  SentimentScore
0               1       Yes             0.93             0.0
1               2        No             0.00             0.0
2               4       Yes             0.94             0.0
3               5        No             0.02             0.0
4               7        No             0.02             0.0


In [16]:
# First 5 rows
print(df.head())

   Age Attrition     BusinessTravel  DailyRate              Department  \
0   41       Yes      Travel_Rarely       1102                   Sales   
1   49        No  Travel_Frequently        279  Research & Development   
2   37       Yes      Travel_Rarely       1373  Research & Development   
3   33        No  Travel_Frequently       1392  Research & Development   
4   27        No      Travel_Rarely        591  Research & Development   

   DistanceFromHome  Education EducationField  EmployeeCount  EmployeeNumber  \
0                 1          2  Life Sciences              1               1   
1                 8          1  Life Sciences              1               2   
2                 2          2          Other              1               4   
3                 3          4  Life Sciences              1               5   
4                 2          1        Medical              1               7   

   ...  TrainingTimesLastYear WorkLifeBalance  YearsAtCompany  \
0  ...   

In [17]:
# Checking for null values
df.isna().sum()

Age                            0
Attrition                      0
BusinessTravel                 0
DailyRate                      0
Department                     0
DistanceFromHome               0
Education                      0
EducationField                 0
EmployeeCount                  0
EmployeeNumber                 0
EnvironmentSatisfaction        0
Gender                         0
HourlyRate                     0
JobInvolvement                 0
JobLevel                       0
JobRole                        0
JobSatisfaction                0
MaritalStatus                  0
MonthlyIncome                  0
MonthlyRate                    0
NumCompaniesWorked             0
Over18                         0
OverTime                       0
PercentSalaryHike              0
PerformanceRating              0
RelationshipSatisfaction       0
StandardHours                  0
StockOptionLevel               0
TotalWorkingYears              0
TrainingTimesLastYear          0
WorkLifeBa

## ETL Pipeline - Loading Data to MYSQL Database

In [21]:
import urllib.parse 
from sqlalchemy import create_engine, text 
# Create Connection 
raw_password = "YOUR_PASSWORD_HERE"
safe_password = urllib.parse.quote_plus(raw_password)
connection_string = "mysql+pymysql://root:{}@localhost:3306/hr_project".format(safe_password)
engine = create_engine(connection_string)

# Test Connection and Load 
try:
    with engine.begin() as conn:  
        print("...Connection successful! Transaction started.")
        
        print("...Disabling foreign key checks.")
        conn.execute(text("SET FOREIGN_KEY_CHECKS=0;"))

        print("...Truncating tables.")
        conn.execute(text("TRUNCATE TABLE ml_scores;"))
        conn.execute(text("TRUNCATE TABLE employee_data;"))

        print("...Preparing DataFrames.")
        df_employee_data = df[[
            'EmployeeNumber', 'Age', 'Attrition', 'Department', 
            'JobRole', 'MonthlyIncome', 'PerformanceRating', 'YearsAtCompany',
            'EnvironmentSatisfaction', 'JobSatisfaction', 'WorkLifeBalance'
        ]]
        df_ml_scores = df[[
            'EmployeeNumber', 'FlightRiskScore', 'SentimentScore', 'ExitCommentText'
        ]]

        print("...Appending to Employee_Data ({} rows).".format(len(df_employee_data)))
        df_employee_data.to_sql('employee_data', con=conn, if_exists='append', index=False)
        
        print("...Appending to ML_Scores ({} rows).".format(len(df_ml_scores)))
        df_ml_scores.to_sql('ml_scores', con=conn, if_exists='append', index=False)
        
        print("...Re-enabling foreign key checks.")
        conn.execute(text("SET FOREIGN_KEY_CHECKS=1;"))

        print("\n--- PYTHON SCRIPT COMPLETE! ---")
        print("Transaction will now be committed (saved).")

except Exception as e:
    print("\n--- TRANSACTION FAILED, ROLLED BACK ---")
    print("Error: {}".format(e))

...Connection successful! Transaction started.
...Disabling foreign key checks.
...Truncating tables.
...Preparing DataFrames.
...Appending to Employee_Data (1470 rows).
...Appending to ML_Scores (1470 rows).
...Re-enabling foreign key checks.

--- PYTHON SCRIPT COMPLETE! ---
Transaction will now be committed (saved).
